In [1]:
import pandas as pd
import numpy
from __future__ import division

In [2]:
df2_WS2T = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/TimeStamp_Engineering/engineering_AS_segment2_workflowS2_has_duration.csv",engine='python')

In [3]:
df2_WS2T.head(n=1)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,time_used_str,time_used_second
0,722111|418134|648088,Send for Assessment,Send for Assessment,26/11/2018 16:59:07,4 days 17 hours 40 minutes 41 seconds,409241.0


In [4]:
df2_WS2T['WRKFLW_TSK_MAP_GROUPED'].nunique()

53

In [5]:
df2_WS2T['WRKFLW_TSK_MAP_DETAILED'].nunique()

80

In [6]:
wd_count = df2_WS2T.groupby('WRKFLW_TSK_MAP_DETAILED').WRKFLW_TSK_MAP_DETAILED.count()
wd_dict = dict()
for i, v in wd_count.items():
    wd_dict[i] = v
sorted_wd_dict = sorted(wd_dict.items(), key = lambda x: x[1], reverse = True)
print(sorted_wd_dict)

[('Send for Assessment', 1901), ('Assessment Decision - Qualified', 1731), ('Assessment Outcome - Successful', 1111), ('Assessment Decision - No Decision', 503), ('Assessment Outcome - Not Successful', 302), ('Acacdemic - Assessment Decision - Qualified', 283), ('OC - Created - Evidence of course completion', 258), ('End My Assessment', 256), ('Assessment Decision - Not Qualified', 246), ('Put Assessment On-Hold', 235), ('Release Assessment Hold', 199), ('Assessment Outcome - Higher Preference Successful', 169), ('Acacdemic - Assessment Decision - Not Qualified', 115), ('AR - Satisfied - Academic results for prior tertiary studies', 93), ('Back to Triage', 83), ('AR - Created - Assessment information (Coursework) (1)', 52), ('Put Triage On-Hold', 49), ('AR - Created - University of Melbourne Statement of Results / transcript', 40), ('Release Triage Hold', 40), ('AR - Satisfied - University of Melbourne Statement of Results / transcript', 39), ('Acacdemic - Assessment Decision - No Deci

In [7]:
list = df2_WS2T.WRKFLW_TSK_MAP_DETAILED.unique()
avg_time_dict = dict()
for key in list :
    df_slides = df2_WS2T[df2_WS2T['WRKFLW_TSK_MAP_DETAILED'] == key]
    sum = df_slides['time_used_second'].sum()
    c = len(df_slides)
    avg = sum//c
    avg_time_dict[key] = avg

sorted_avg_time_dict = sorted(avg_time_dict.items(),key = lambda x: x[1], reverse = True)
print(sorted_avg_time_dict)

[('Put Assessment On-Hold', 1804236.0), ('AR - Satisfied - University of Melbourne Statement of Results / transcript', 1222245.0), ('AR - Created - Specialisation (Master of Engineering)', 1214250.0), ('End My Assessment', 1118958.0), ('Send for Assessment', 1004177.0), ('Put Triage On-Hold', 980514.0), ('AR - Satisfied - Entry points (Master of Information Systems)', 959064.0), ('AR - Satisfied - Official syllabus or subject descriptions (in English)', 935013.0), ('Acacdemic - Assessment Decision - No Decision', 888731.0), ('AR - Satisfied - Credit for previous study', 842830.0), ('AR - Created - Assessment information (Coursework) (1)', 775549.0), ('AR - Satisfied - Assessment information (Coursework) (1)', 760815.0), ('AR - Satisfied - Full application for Coursework program (1)', 641560.0), ('AR - Satisfied - Academic results for prior tertiary studies', 622999.0), ('OC - Satisfied - English language requirements', 605709.0), ('Assessment Outcome - Ineligible / Withdrawn', 591080.0

In [8]:
numOfWorkflowProcesses = df2_WS2T.groupby('CASE_ID').CASE_ID.count()

count_dic=dict()
for i,v in numOfWorkflowProcesses.items():
    if count_dic.get(v) is None:
        count_dic[v]=[]
        count_dic[v].append(i)
    else:
        count_dic[v].append(i)
#count_dic
count_dic = dict(sorted(count_dic.items()))

In [9]:
len_list=[]
for key in count_dic.keys():
    len_list.append(len(count_dic[key]))

In [10]:
def strfdelta(tdelta, fmt):
    d=dict()
    d["days"],rem1 = divmod(tdelta,60*60*24)
    d["hours"], rem2 = divmod(rem1, 3600)
    d["minutes"], d["seconds"] = divmod(rem2, 60)
    return fmt.format(**d)

In [11]:
list_avgtime_per_key =[]
list_sumtime_row = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_person = 0
    for id in id_list:
        df = df2_WS2T[df2_WS2T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_person+=tsum
    list_avgtime_per_key.append(int((list_sumtime_person/len(count_dic[key]))))
    list_sumtime_row.append(list_sumtime_person)
list_avgtime_per_key_str = []
for i in list_avgtime_per_key:
    list_avgtime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [12]:
list_mediantime_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_per_person = []
    for id in id_list:
        df = df2_WS2T[df2_WS2T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_per_person.append(tsum)
    list_mediantime_per_key.append(numpy.median(list_sumtime_per_person))

list_mediantime_per_key_str = []
for i in list_mediantime_per_key:
    list_mediantime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [13]:
list_percent_reach_end = []
for key in count_dic.keys():
    id_list = count_dic[key]
    count_df_end_step = 0
    count_df_id = 0
    for id in id_list:
        df_id = df2_WS2T[df2_WS2T['CASE_ID']== id]
        df_end_step = df_id[df_id['WRKFLW_TSK_MAP_GROUPED'].isin(['Assessment Outcome - Successful','Assessment Outcome - Not Successful']) ]
        count_df_id += df_id['CASE_ID'].nunique()
        count_df_end_step += df_end_step['CASE_ID'].nunique()
    list_percent_reach_end.append(str(round((count_df_end_step/count_df_id)*100,2))+ "%")

In [14]:
list_step_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = set()
    for id in id_list:
        df = df2_WS2T[df2_WS2T['CASE_ID']== id]
        for index, row in df.iterrows():
            list_step.add(row['WRKFLW_TSK_MAP_GROUPED'])
    list_step_per_key.append(list_step)

In [15]:
from collections import Counter
import collections, functools, operator
list_step_sum_time_per_key = []
list_step_count_per_key = []
list_step_ave_time_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = []
    step_time_list = []
    for id in id_list:
        df = df2_WS2T[df2_WS2T['CASE_ID']== id]
        step_time = dict(zip(df.WRKFLW_TSK_MAP_GROUPED, df.time_used_second))
        step_time_list.append(step_time)
        for index, row in df.iterrows():
            list_step.append(row['WRKFLW_TSK_MAP_GROUPED'])

    step_sum_time = dict(functools.reduce(operator.add,
         map(collections.Counter, step_time_list)))

    dict_list_step=dict(Counter(list_step))

    step_ave_time = {key: strfdelta(step_sum_time[key] // dict_list_step.get(key, 0),"{days} days {hours} hours {minutes} minutes {seconds} seconds")
                        for key in step_sum_time.keys()}

    sort_step_sum_time = sorted(step_sum_time.items(),key = lambda x: x[1], reverse = True)
    list_step_sum_time_per_key.append(sort_step_sum_time)

    sort_dict_list_step = sorted(dict_list_step.items(),key = lambda x: x[1], reverse = True)
    list_step_count_per_key.append(sort_dict_list_step[:6])

    sort_step_ave_time = sorted(step_ave_time.items(),key = lambda x: x[1], reverse = True)
    list_step_ave_time_per_key.append(sort_step_ave_time)


In [16]:
s2_t2=pd.DataFrame({'Number of Workflow processes in this Workflow Segment':count_dic.keys(),'Number of cases':len_list, 'average time for workflow segment':list_avgtime_per_key_str,'median time for workflow segment':list_mediantime_per_key_str,'percentage of cases reaching to end of workflow segment':list_percent_reach_end})
s2_t2['Num of Occurrences per step(descending)'] = list_step_count_per_key
s2_t2['Average time spends per step(descending)'] = list_step_ave_time_per_key
s2_t2['average time for workflow segment(seconds)'] = list_avgtime_per_key
s2_t2['sum time for workflow segment(seconds)'] = list_sumtime_row
s2_t2.head(n=10)


,Number of Workflow processes in this Workflow Segment,Number of cases,average time for workflow segment,median time for workflow segment,percentage of cases reaching to end of workflow segment,Num of Occurrences per step(descending),Average time spends per step(descending),average time for workflow segment(seconds),sum time for workflow segment(seconds)
0,1,115,0 days 0 hours 0 minutes 0 seconds,0.0 days 0.0 hours 0.0 minutes 0.0 seconds,0.0%,"[(Send for Assessment, 115)]",[],0,0.0
1,2,166,19 days 10 hours 41 minutes 41 seconds,11.0 days 1.0 hours 2.0 minutes 35.0 seconds,35.54%,"[(Send for Assessment, 166), (Assessment Outco...","[(Send for Assessment, 19.0 days 10.0 hours 41...",1680101,278896841.0
2,3,563,19 days 13 hours 37 minutes 41 seconds,14.0 days 4.0 hours 44.0 minutes 39.0 seconds,91.65%,"[(Send for Assessment, 563), (Assessment Outco...",[(Acacdemic - Assessment Decision - Not Qualif...,1690661,951842582.0
3,4,331,28 days 3 hours 10 minutes 55 seconds,19.0 days 6.0 hours 45.0 minutes 19.0 seconds,85.2%,"[(Send for Assessment, 335), (Assessment Decis...","[(AR - Satisfied - advanced standing, 9.0 days...",2430655,804547109.0
4,5,219,33 days 19 hours 51 minutes 14 seconds,20.0 days 23.0 hours 44.0 minutes 56.0 seconds,87.21%,"[(Assessment Decision - Qualified, 265), (Send...","[(AR - Not Satisfied - ELR, 8.0 days 18.0 hour...",2922674,640065608.0
5,6,156,35 days 21 hours 27 minutes 16 seconds,26.0 days 12.0 hours 33.0 minutes 26.0 seconds,85.26%,"[(Assessment Decision - Qualified, 219), (Send...","[(AR - Satisfied - entry points, 8.0 days 21.0...",3101236,483792929.0
6,7,77,35 days 16 hours 36 minutes 27 seconds,22.0 days 2.0 hours 16.0 minutes 35.0 seconds,80.52%,"[(Assessment Decision - Qualified, 100), (Send...","[(Put Assessment On-Hold, 7.0 days 18.0 hours ...",3083787,237451671.0
7,8,54,48 days 17 hours 13 minutes 39 seconds,29.0 days 23.0 hours 18.0 minutes 29.0 seconds,83.33%,"[(Assessment Decision - Qualified, 93), (Send ...","[(AR - Satisfied - syllabus, 9.0 days 20.0 hou...",4209219,227297830.0
8,9,39,42 days 22 hours 37 minutes 14 seconds,34.0 days 18.0 hours 35.0 minutes 19.0 seconds,82.05%,"[(Assessment Decision - Qualified, 76), (Send ...","[(Send for Assessment, 7.0 days 10.0 hours 44....",3710234,144699164.0
9,10,22,72 days 16 hours 30 minutes 40 seconds,23.0 days 10.0 hours 23.0 minutes 5.5 seconds,81.82%,"[(Assessment Decision - Qualified, 37), (Asses...","[(AR - Satisfied - GAM, 8.0 days 23.0 hours 29...",6280240,138165296.0


In [17]:
s2_t2.to_csv('engineering_AS_segment2_workflowS2_table.csv', index=False)






